In [1]:
using Pkg
Pkg.activate("F:/Tue/PhD/GP_playgroundcode")

  Activating project at `F:\Tue\PhD\GP_playgroundcode`


In [2]:
using ReactiveMP, KernelFunctions, Distributions, LinearAlgebra,GraphPPL, Rocket,Plots,Revise 
using Flux, Zygote, ForwardDiff
include("utils_gp2.jl")
pgfplotsx()
import StatsFuns: log2π

In [3]:
xmin, xmax = -3, 3  # Bounds of the data
N = 200# Number of samples
xtrain = sort(rand(Uniform(xmin, xmax), N))
xtest = collect(range(xmin - 0.1, xmax + 0.1; length=600));
xtest = sort(vcat(xtest,xtrain))
precision = 100
ytrain = sinc.(xtrain) + randn(N) * sqrt(1/precision);
f_true = sinc.(xtest);

In [4]:
# kernelfunc(θ) = with_lengthscale(Matern52Kernel(),θ)

kernelfunc (generic function with 1 method)

In [22]:
# test new thing
@model [ default_factorisation = FullFactorisation() ] function gpprocess(n,kernelfunc,meanfunc,train,test)
    y = datavar(Float64,n)
    x = randomprocess(test,train)
    ### change here 
    θ = randomvar() where { prod_constraint = ProdGeneric(), marginal_form_constraint = PointMassFormConstraint() }
    θ ~ NormalMeanVariance(0.7, 1.)
    x ~ GaussianProcess(meanfunc,kernelfunc, θ)
    #######
    γ ~ GammaShapeRate(1.0,0.1)
    for i=1:n 
        y[i] ~ NormalMeanPrecision(x, γ) where {meta=ProcessMeta(i,nothing,nothing,nothing), q = q(x)q(γ)q(y[i])}
    end
    return x, γ, θ, y  
end

In [23]:
function Distributions.entropy(pm::PointMass{F}) where {F <: Function}
    return ReactiveMP.InfCountingReal(Float64,-1)
end

function Distributions.entropy(pm::PointMass{F}) where {F <: Kernel}
    return ReactiveMP.InfCountingReal(Float64,-1)
end

###changed 
@average_energy GaussianProcess (q_out::GaussianProcess, q_meanfunc::Any, q_kernelfunc::Any, q_θ::Any) = begin
    q_out.finitemarginal
    return -entropy(q_out.finitemarginal)
end
############

@average_energy NormalMeanPrecision (q_out::Any, q_μ::GaussianProcess, q_τ::Any,meta::ProcessMeta) = begin
    m_right, cov_right = mean_cov(q_μ.finitemarginal)
    kernelf = q_μ.kernelfunction
    meanf   = q_μ.meanfunction
    test    = q_μ.testinput
    train   = q_μ.traininput
    μ_mean, μ_var = predMVN(q_μ,test,[train[meta.index]],m_right)
    μ_var = clamp(μ_var[1],1e-8,huge)
    μ_mean = μ_mean[1]
    out_mean, out_var = mean_var(q_out)
    return (log2π - mean(log, q_τ) + mean(q_τ) * (μ_var + out_var + abs2(μ_mean - out_mean))) / 2
end

function ReactiveMP.entropy(p::GaussianProcess)
    return ReactiveMP.entropy(p.finitemarginal)
end

@rule NormalMeanPrecision(:μ, Marginalisation) (q_out::PointMass, q_τ::GammaShapeRate, meta::ProcessMeta) = begin 
    return @call_rule NormalMeanPrecision(:μ, Marginalisation) (q_out=q_out,q_τ=q_τ,meta=nothing)
end
# @rule GaussianProcess(:out, Marginalisation) (q_meanfunc::PointMass, q_kernelfunc::PointMass) = begin 
#     return GaussianProcess(q_meanfunc.point,q_kernelfunc.point,nothing,nothing,nothing,nothing)
# end

In [24]:
kernel_func =  Matern52Kernel() 
meanfunc   = (x) -> 0.0;

In [25]:
# This is the product function that computes the product of messages for GP node.
# function ReactiveMP.messages_prod_fn(process::RandomProcess)
#     test  = process.test_input
#     train = process.train_input
#     # @show fieldnames(typeof(process))
#     return messages -> begin 
#         m_right,cov_right = make_multivariate_message(map(as_message, messages))

#         return Message(MvNormalMeanCovariance(m_right, cov_right), false, false)
#     end
# end

function ReactiveMP.messages_prod_fn(process::RandomProcess)
    return ReactiveMP.marginal_prod_fn(process)
end

function ReactiveMP.marginal_prod_fn(process::RandomProcess)
    test  = process.test_input
    train = process.train_input

    return messages -> begin 
        message_vector = map(ReactiveMP.as_message, messages)
        process_message = getdata(message_vector[1])
        meanf = process_message.meanfunction
        kernelf = process_message.kernelfunction
        likelihood_messages = message_vector[2:end]
        m_right,cov_right = make_multivariate_message(likelihood_messages)
        
        #m, K = predMVN(meta,process_message,train, test, m_right,cov_right)
        m, K= predMVN(kernelf,meanf,train,test,m_right,cov_right) #Ismail code
        Kff = kernelmatrix(kernelf, test, test)
        invKff = cholinv(Kff + K)
        return Marginal(GaussianProcess(meanf,kernelf,MvNormalMeanCovariance(m,K),test,train, invKff),false,false)
    end
end

In [26]:
@constraints function gpconstraints()   
    q(θ) :: PointMass(starting_point = (args...) -> [ 0.5 ])
    q(x,γ) = q(x)q(γ)
end

gpconstraints (generic function with 1 method)

In [27]:
nits = 1
iresult = inference(
    model = Model(gpprocess, length(ytrain), kernel_func, meanfunc, xtrain,xtest),
    initmarginals = (γ = vague(GammaShapeRate),),
    options = model_options(limit_stack_depth = 500),
    # constraints = gpconstraints(),
    iterations = nits,  
    data  = (y = ytrain,),
    showprogress=true,
    free_energy = true
)


┌ Error: Error showing method candidates, aborted
│   exception = (UndefRefError(), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000002733fce, Ptr{Nothing} @0x0000000002700e9c, Ptr{Nothing} @0x0000000000d3b45c, Ptr{Nothing} @0x000000009e6d53ec, Ptr{Nothing} @0x000000009e6d8f20, Ptr{Nothing} @0x000000009e6d9424, Ptr{Nothing} @0x000000009e6c764a, Ptr{Nothing} @0x000000009e6c7813, Ptr{Nothing} @0x000000009e6c7884, Ptr{Nothing} @0x00000000026fd1b6, Ptr{Nothing} @0x000000001c9fa1c3, Ptr{Nothing} @0x000000001c9faace, Ptr{Nothing} @0x000000001c9fad8e, Ptr{Nothing} @0x000000001c9fae04, Ptr{Nothing} @0x000000001c9eeffa, Ptr{Nothing} @0x000000001c9e43c1, Ptr{Nothing} @0x000000001c9e4556, Ptr{Nothing} @0x000000001c9e45c1, Ptr{Nothing} @0x000000001c9deac5, Ptr{Nothing} @0x000000000272a48f, Ptr{Nothing} @0x000000000272ac41, Ptr{Nothing} @0x000000000272bcaf, Ptr{Nothing} @0x0000000010a18f51, Ptr{Nothing} @0x0000000010961205, Ptr{Nothing} @0x000000001031fb58, Ptr{Nothing} @0x0000000010

MethodError: MethodError: no method matching make_node(::Type{GaussianProcess}, ::FactorNodeCreationOptions{FullFactorisation, Nothing, Nothing}, ::RandomProcess, ::Array{ConstVariable{PointMass{Float64}, SingleObservable{Message{PointMass{Float64}}, AsapScheduler}}, 0}, ::ConstVariable{Matern52Kernel{Distances.Euclidean}, SingleObservable{Message{Matern52Kernel{Distances.Euclidean}}, AsapScheduler}}, ::RandomVariable)